## [결측치 Missing Value 처리]
### - 데이터 미입력 또는 데이터 저장 과정에서 지워진 데이터
### - 빈칸 의미
### - 표시 : NaN (Not A Number) NaT (Not A Text)
### - 표현 : numpy 모듈의 nan, math 모듈의 nan 사용

### (1) 모듈 로딩

In [2]:
import pandas as pd

### (2) 데이터 준비

In [3]:
file = '../DATA/employees.csv'

### (3) 데이터 저장 : CSV => DataFrame

In [4]:
empDF=pd.read_csv(file)

In [5]:
empDF


,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,8/6/93,NaN,True,Marketing
1,Thomas,Male,3/31/96,61933.0,True,NaN
2,Maria,Female,NaN,130590.0,False,Finance
3,Jerry,NaN,3/4/05,138705.0,True,Finance
4,Larry,Male,1/24/98,101004.0,True,IT
...,...,...,...,...,...,...
996,Phillip,Male,1/31/84,42392.0,False,Finance
997,Russell,Male,5/20/13,96914.0,False,Product
998,Larry,Male,4/20/13,60500.0,False,Business Dev
999,Albert,Male,5/15/12,129949.0,True,Sales


### (4) 데이터 확인

In [6]:
# 데이터의 전반적인 요약정보 확인 => info() 메서드
# - 컬럼별 결측치 여부 확인 ==> 실제 데이터에서 결측치 체크 & 처리
# - 컬럼별 데이터 타입과 실제 데이터 타입 비교 ==> 타입 다른 경우, 타입 변환
empDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   First Name  933 non-null    object 
 1   Gender      854 non-null    object 
 2   Start Date  999 non-null    object 
 3   Salary      999 non-null    float64
 4   Mgmt        933 non-null    object 
 5   Team        957 non-null    object 
dtypes: float64(1), object(5)
memory usage: 47.0+ KB


### - 데이터의 컬럼별 분포 확인 => describe() : 컬럼별 기술통계값

In [7]:
empDF.describe()

,Salary
count,999.000000
mean,90655.528529
std,32939.511615
min,35013.000000
25%,62560.000000
50%,90427.000000
75%,118744.500000
max,149908.000000


In [8]:
# 최빈값 => mode(), 중앙값 => median()
print(empDF.mode(numeric_only=True),
      empDF.median(numeric_only=True),
      empDF.min(numeric_only=True),
      sep='\n\n')

     Salary
0   86676.0
1   91462.0
2  121160.0
3  145988.0
4  147183.0

Salary    90427.0
dtype: float64

Salary    35013.0
dtype: float64


### -(5) 데이터 전처리(Preprocessing)=> 가. 결측치 처리

### - 결측치 확인 ==> isna(), isnull() : 원소 단위로 검사 후 결측치인 경우

In [9]:
empDF.columns

Index(['First Name', 'Gender', 'Start Date', 'Salary', 'Mgmt', 'Team'], dtype='object')

In [10]:
# 전체 데이터의 컬럼별 결측치 체크
empDF.isna()
empDF.isnull()

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,False,False,False,True,False,False
1,False,False,False,False,False,True
2,False,False,True,False,False,False
3,False,True,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
996,False,False,False,False,False,False
997,False,False,False,False,False,False
998,False,False,False,False,False,False
999,False,False,False,False,False,False


In [11]:
empDF.isnull().sum()

First Name     68
Gender        147
Start Date      2
Salary          2
Mgmt           68
Team           44
dtype: int64

### - 결측치 확인 ==> notna() / notnull() : 결측치가 아니면 True 반환

In [12]:
print(empDF.notna().head(), empDF.notnull().sum(), sep='\n\n')

   First Name  Gender  Start Date  Salary  Mgmt   Team
0        True    True        True   False  True   True
1        True    True        True    True  True  False
2        True    True       False    True  True   True
3        True   False        True    True  True   True
4        True    True        True    True  True   True

First Name    933
Gender        854
Start Date    999
Salary        999
Mgmt          933
Team          957
dtype: int64


### - 결측치 처리 ==> (1) 삭제 : dropna()

In [13]:
# [기본] 모든 행에 1개의 값이라도 NaN이면 삭제
empDF2=empDF.dropna()
print(empDF.shape, empDF2.shape)

(1001, 6) (761, 6)


In [14]:
# [설정] 행의 모든 데이터 즉 모든 값이 NA면 삭제
# 
empDF3=empDF.dropna(how='all')
print(empDF.shape, empDF3.shape)

(1001, 6) (1000, 6)


In [15]:
# [설정] 특정 컬럼의 NA만 체크 후 삭제하는 방법
# 예) 성별에 따른 연봉 분석 => NA 면 분석 불가능 컬럼을 지정
empDF4=empDF.dropna(subset=['Gender','Salary'])
print(empDF.shape, empDF4.shape)

(1001, 6) (853, 6)


In [16]:
# [설정] 컬럼별 정상 데이터 즉 NA가 아닌 데이터의 갯수를 지정 후
#        해당 수 만큼 정상 데이터가 없는 컬럼 삭제
empDF5 = empDF.dropna(thresh=900, axis='columns')
print(empDF.shape, empDF5.shape, empDF.columns)

(1001, 6) (1001, 5) Index(['First Name', 'Gender', 'Start Date', 'Salary', 'Mgmt', 'Team'], dtype='object')


### - 결측치 처리 => 대체 : fillna()

In [17]:
# Gender 컬럼에 결측 값은 무엇으로 채울까???
empDF.describe(include='all').head(4)

,First Name,Gender,Start Date,Salary,Mgmt,Team
count,933,854,999,999.0,933,957
unique,200,2,971,NaN,2,10
top,Marilyn,Female,10/30/04,NaN,True,IT
freq,11,431,2,NaN,468,106


In [18]:
# ==> 컬럼의 최빈값으로 na 채우기
# empDF.Gender.mode()[0]
genSR = empDF['Gender'].fillna(empDF.Gender.mode()[0])
genSR.isna().sum()
import numpy as np
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                [3, 4, np.nan, 1],
                [np.nan, np.nan, np.nan, np.nan],
                [np.nan, 3, np.nan, 4]],
               columns=list("ABCD"))

In [19]:
# ==> NA값 이전과 이후 값으로 채우기 => method = 'ffill', method = 'bfill'
empDF.Gender.fillna(method='ffill')

0         Male
1         Male
2       Female
3       Female
4         Male
         ...  
996       Male
997       Male
998       Male
999       Male
1000      Male
Name: Gender, Length: 1001, dtype: object

In [20]:
import numpy as np
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                    [3, 4, np.nan, 1],
                    [np.nan, np.nan, np.nan, np.nan],
                    [np.nan, 3, np.nan, 4]],
                   columns=list("ABCD"))

In [21]:
df

,A,B,C,D
0,NaN,2.0,NaN,0.0
1,3.0,4.0,NaN,1.0
2,NaN,NaN,NaN,NaN
3,NaN,3.0,NaN,4.0


In [22]:
df.A.fillna(method='ffill', limit=1)

0    NaN
1    3.0
2    3.0
3    NaN
Name: A, dtype: float64

In [23]:
df.A.fillna(method='bfill', limit=1)


0    3.0
1    3.0
2    NaN
3    NaN
Name: A, dtype: float64

In [24]:
# B 컬럼과 D 컬럼을 평균으로 결측치를 대체
df.B.fillna(df.B.mean())

0    2.0
1    4.0
2    3.0
3    3.0
Name: B, dtype: float64

In [25]:
# B 컬럼과 D 컬럼을 최빈값으로 결측치를 대체
df.B.fillna(df.B.median())

0    2.0
1    4.0
2    3.0
3    3.0
Name: B, dtype: float64